In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import opendatasets as od

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader


In [4]:
od.download('https://www.kaggle.com/datasets/uciml/iris',data_dir='./datasets/')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading iris.zip to ./datasets/iris


100%|██████████| 3.60k/3.60k [00:00<00:00, 80.3kB/s]

In [9]:
df = pd.read_csv('./datasets/iris/Iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [11]:
df = df.drop('Id',axis=1)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [18]:
df = df.iloc[:100]
df['Species'] = df['Species'].apply(lambda x: 0 if x == 'Iris-setosa' else 1)
print(df.head(100))

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  Species
0             5.1           3.5            1.4           0.2        0
1             4.9           3.0            1.4           0.2        0
2             4.7           3.2            1.3           0.2        0
3             4.6           3.1            1.5           0.2        0
4             5.0           3.6            1.4           0.2        0
..            ...           ...            ...           ...      ...
95            5.7           3.0            4.2           1.2        1
96            5.7           2.9            4.2           1.3        1
97            6.2           2.9            4.3           1.3        1
98            5.1           2.5            3.0           1.1        1
99            5.7           2.8            4.1           1.3        1

[100 rows x 5 columns]


C:\Users\pkdiv\AppData\Local\Temp\ipykernel_8908\4032063653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Species'] = df['Species'].apply(lambda x: 0 if x == 'Iris-setosa' else 1)


In [22]:
features = torch.tensor(df[['SepalLengthCm','PetalLengthCm']].values,dtype=torch.float32)
labels = torch.tensor(df['Species'].values,dtype=torch.int)

In [24]:
features.shape, labels.shape

(torch.Size([100, 2]), torch.Size([100]))

In [35]:
torch.manual_seed(1)
shuffled_index = torch.randperm(labels.size()[0], dtype=torch.long)

tensor([45, 15, 90, 32, 35, 63, 17, 72, 79, 96, 48, 36, 16, 11, 23, 80, 22, 58,
         3, 62, 50, 33, 66, 99, 43, 76,  7, 57, 81, 82,  6, 10, 24, 52, 95, 73,
        91, 21, 38, 31, 85, 59, 13, 69, 75, 70, 64,  8, 77, 34, 46, 39, 92,  0,
        44, 98, 49,  9,  4, 61, 12, 83, 28, 78, 40, 88, 54,  5, 26, 41, 89, 20,
        84,  2,  1, 55, 19, 74, 25, 37, 42, 14, 30, 18, 67, 71, 68, 27, 60, 51,
        29, 56, 93, 47, 97, 94, 86, 87, 65, 53])

In [36]:
features,labels = features[shuffled_index],labels[shuffled_index]

In [42]:
X_train, X_test = features[shuffled_index[:75]],features[shuffled_index[75:]]
y_train,y_test = labels[shuffled_index[:75]],labels[shuffled_index[75:]]

In [51]:
mean = X_train.mean(dim=0)
std = X_train.std(dim=0)

X_train = (X_train-mean)/std
X_test = (X_test-mean)/std